In [1]:
import tensorflow as tf
import pandas as pd

#### calc vector position of 1 for all words in word column

In [5]:
df = pd.read_csv("db/sample/clean/final.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])

df_nums = pd.DataFrame(columns=["word","number"], index=None)
count = 0;

def word_vector_nums(row):
    global count, df_nums
    df_nums = df_nums.append({"word":row.word,"number": count}, ignore_index=True)
    count += 1
    return ""

df.apply(word_vector_nums, axis=1)
df_nums.to_csv("db/sample/clean/word_col_vectors.csv", sep=":", columns=["word","number"], header=False, index=False, mode="w")

### convert definitions to vector positions

In [6]:
df = pd.read_csv("db/sample/clean/final.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
df_words = pd.read_csv("db/sample/clean/word_col_vectors.csv", names=["word","number"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
df_defs = pd.DataFrame(columns=["word", "def_vectors"], index=None)

df_2 = df[:6575]

def def_vector_nums(row):
    global df_defs
    definition = str(row.definition)
    if definition is not None:
        vector =  " ".join(str(df_words.loc[df_words["word"] == word]["number"].values[0]) for word in definition.split())
        df_defs = df_defs.append({"word":row.word, "def_vectors":vector}, ignore_index=True)
    return

df_2.apply(def_vector_nums, axis=1)
df_defs.to_csv("db/sample/clean/def_col_vectors.csv", sep=":", columns=["word","number"], header=False, index=False, mode="w")

C:\Users\ritiksax\AppData\Local\Continuum\miniconda3\envs\revdict\lib\site-packages\pandas\io\formats\format.py:1586: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  self.obj = self.obj.loc[:, cols]
C:\Users\ritiksax\AppData\Local\Continuum\miniconda3\envs\revdict\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [ ]:
df_2 = pd.DataFrame(columns=["word"],index=None)
df_2 = df[6576:]